In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,227 kB]
Get:14 http://security.ubuntu.com/

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
#Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

df.show(5)

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date=datetime.date(2022, 4, 8), date_built=2016, price=936923, bedrooms=4, bathrooms=3, sqft_living=3167, sqft_lot=11733, floors=2, waterfront=1, view=76)

In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_4bed = """
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    YEAR(date)
ORDER BY
    year
"""

# Execute the query and show results
result = spark.sql(avg_price_4bed)
result.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3bed_3bath = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query and show results
result = spark.sql(avg_price_3bed_3bath)
result.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_3bed_3bath_2flr_2000sqft = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
WHERE
    bedrooms = 3
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Execute the query and show results
result = spark.sql(avg_price_3bed_3bath_2flr_2000sqft)
result.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [11]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Start the timer
start_time = time.time()

# Create the query
avg_price_per_viewrating_gt_350000 = """
SELECT
    view,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

# Execute the query and show results
result = spark.sql(avg_price_per_viewrating_gt_350000)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9607670307159424 seconds ---


In [12]:
# 7. Cache the temporary table home_sales
spark.sql("CACHE TABLE home_sales")

+--------+
|count(1)|
+--------+
|   33287|
+--------+



In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_price_per_viewrating_gt_350000 = """
SELECT
    view,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""
result = spark.sql(avg_price_per_viewrating_gt_350000)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.7178351879119873 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [17]:
# 11. Read the parquet formatted data.
df_partitioned = spark.read.parquet("home_sales_partitioned")

In [18]:
# 12. Create a temporary table for the parquet data.
df_partitioned.createOrReplaceTempView("home_sales_partitioned")

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales_partitioned
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view DESC
"""

result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9412796497344971 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

print(f"Is home_sales still cached? {is_cached}")

Is home_sales still cached? False
